In [4]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE
using FUSE.IMAS

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [18]:
R0 = 1.8
δ = 0.5
ϵ = 0.32
κ = 1.7
B0 = 2.0
qstar = 1.57
beta_t = 0.1

eq0=FUSE.init(IMAS.equilibrium(), 0.0; B0, R0, ϵ, δ, κ, beta_t, qstar)

@show eq0

@time tmp=FUSE.SolovevEquilibriumActor(eq0, 0.0; verbose=false)

time_slice
｜[1]
｜｜profiles_1d
｜｜｜elongation ➡ 1-element Vector{Float64}
｜｜｜psi ➡ 1-element Vector{Float64}
｜｜｜triangularity_upper ➡ 1-element Vector{Float64}
｜｜｜r_outboard ➡ 1-element Vector{Float64}
｜｜｜triangularity_lower ➡ 1-element Vector{Float64}
｜｜｜r_inboard ➡ 1-element Vector{Float64}
｜｜｜q ➡ 1-element Vector{Float64}
｜｜global_quantities
｜｜｜beta_tor ➡ Float64
｜｜｜energy_mhd ➡ IMAS.var"#10#12"
time ➡ 1-element Vector{Float64}
vacuum_toroidal_field
｜b0 ➡ 1-element Vector{Float64}
｜r0 ➡ Float64
eq0 = 
  0.361191 seconds (5.83 M allocations: 136.900 MiB, 4.56% gc time)


SolovevEquilibriumActor(SolovevEquilibrium
  B0 = 2.0 [T]
  R0 = 1.8 [m]
  ϵ  = 0.32000000000000006
  δ  = 0.5
  κ  = 1.7
  α  = -0.9299145920430387
  q⋆ = 1.57
  βp = 2.3844963176153855
  βt = 0.0990597683166926
  σ  = 1
  diverted  = false
  symmetric = true)